# Zanesville OH Web Scraping Demo

Below gives an example, using a couple of parcel numbers, of the process necessary to scrape the Muskingum County Auditor's website for housing data.

In [127]:
import os
import re
import numpy as np
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
from geopy.distance import great_circle

In [2]:
PROJECT_PATH = os.path.join(os.getcwd(), '..')

geolocator = Nominatim()
WAREHOUSE_ADDRESS_DICT = {
    'street': '1525 Pershing Road',
    'city': 'Zanesville',
    'state': 'Ohio',
    'postalcode': '43701'
}
WAREHOUSE_GEOLOCATION = geolocator.geocode(WAREHOUSE_ADDRESS_DICT)
WAREHOUSE_LATLONG = (WAREHOUSE_GEOLOCATION.latitude, WAREHOUSE_GEOLOCATION.longitude)

Prepare a dictionary of URLs for future use:

In [3]:
urls = {
    'advanced-search-results': 'http://www.muskingumcountyauditor.org/Results.aspx?SearchType=Advanced&Criteria=20g%2byYTTdkDKRrEbpO1sLV9b36Zp5GCYSiEbzYPtPXU%3d',
    'parcel-id-data-fmt': 'http://muskingumcountyauditor.org/Data.aspx?ParcelID={parcel_id}'
}

Create a webdriver object that will allow us to interact with the website through JavaScript. This is necessary because the auditor's website uses a lot of single page features that don't map to URLs. Thus, we are forced to use JavaScript to push the necessary buttons for scraping our data.

In [4]:
search_page_driver = webdriver.PhantomJS(executable_path=os.path.join(PROJECT_PATH, 'resources', 'phantomjs'))

In [5]:
search_page_driver.get(urls['advanced-search-results'])

In [6]:
search_page_driver.page_source

'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd"><html xmlns="http://www.w3.org/1999/xhtml"><head id="head"><meta http-equiv="X-UA-Compatible" content="IE=Edge"><title>\n\tMuskingum County, Ohio: Online Auditor - Disclaimer\n</title><meta http-equiv="content-type" content="text/html; charset=iso-8859-1"><meta http-equiv="content-language" content="en"><meta http-equiv="pragma" content="no-cache"><meta name="description" content="Online Real Estate and Property Search"><meta name="keywords"><meta name="robots" content="index,follow"><meta name="author" content="Kelly Menzel"><meta name="company" content="Digital Data Technologies, Inc."><meta name="copyright" content="Copyright ©2007"><link href="/favicon.ico" rel="Shortcut Icon"><link href="/stylesheets/print.css" rel="stylesheet" type="text/css" media="print"><link id="stylesheet" rel="stylesheet" type="text/css" href="/stylesheets/Canvas.css"></head>\n    <body 

Here is an example of pushing a button to accept their disclaimer about the quality of the data online:

In [7]:
search_page_driver.execute_script("document.getElementById('ContentPlaceHolder1_btnDisclaimerAccept').click();")

In [9]:
search_page_driver.page_source

'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd"><html xmlns="http://www.w3.org/1999/xhtml"><head id="head"><meta http-equiv="X-UA-Compatible" content="IE=Edge"><title>\n\tMuskingum County, Ohio: Online Auditor - Results\n</title><meta http-equiv="content-type" content="text/html; charset=iso-8859-1"><meta http-equiv="content-language" content="en"><meta http-equiv="pragma" content="no-cache"><meta name="description" content="Online Real Estate and Property Search"><meta name="keywords"><meta name="robots" content="index,follow"><meta name="author" content="Kelly Menzel"><meta name="company" content="Digital Data Technologies, Inc."><meta name="copyright" content="Copyright ©2007"><link href="/favicon.ico" rel="Shortcut Icon"><link href="/stylesheets/print.css" rel="stylesheet" type="text/css" media="print"><link id="stylesheet" rel="stylesheet" type="text/css" href="/stylesheets/Canvas.css"></head>\n    <body id=

Now that we got past the disclaimer, we can use the CSS class names to extract the parcel numbers from the first page:

In [10]:
adv_search_soup = BeautifulSoup(search_page_driver.page_source, 'html5lib')
adv_search_soup.prettify()

'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml">\n <head id="head">\n  <meta content="IE=Edge" http-equiv="X-UA-Compatible"/>\n  <title>\n   Muskingum County, Ohio: Online Auditor - Results\n  </title>\n  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>\n  <meta content="en" http-equiv="content-language"/>\n  <meta content="no-cache" http-equiv="pragma"/>\n  <meta content="Online Real Estate and Property Search" name="description"/>\n  <meta name="keywords"/>\n  <meta content="index,follow" name="robots"/>\n  <meta content="Kelly Menzel" name="author"/>\n  <meta content="Digital Data Technologies, Inc." name="company"/>\n  <meta content="Copyright ©2007" name="copyright"/>\n  <link href="/favicon.ico" rel="Shortcut Icon"/>\n  <link href="/stylesheets/print.css" media="print" rel="stylesheet" type="text/css"/>\n  <link href="/stylesheets/Canvas.css" 

In [11]:
adv_search_soup.find_all('tr', {'class': ['rowstyle', 'alternatingrowstyle']})

[<tr align="left" class="rowstyle" style="border-style:None;">
 			<td style="width:150px;"><a href="javascript:__doPostBack('ctl00$ContentPlaceHolder1$gvSearchResults','Select$0')">17-19-01-04-000</a></td><td>BEAM JEFF R TRUSTEE</td><td>3870  FRAZEYSBURG RD </td><td style="width:100px;">100</td><td style="width:100px;">74.51</td>
 		</tr>,
 <tr align="left" class="alternatingrowstyle" style="border-style:None;">
 			<td style="width:150px;"><a href="javascript:__doPostBack('ctl00$ContentPlaceHolder1$gvSearchResults','Select$1')">17-19-01-04-003</a></td><td>CONDOS AT COLONY HILL</td><td>3721  COLONY HILL DR </td><td style="width:100px;">499</td><td style="width:100px;">0.37</td>
 		</tr>,
 <tr align="left" class="rowstyle" style="border-style:None;">
 			<td style="width:150px;"><a href="javascript:__doPostBack('ctl00$ContentPlaceHolder1$gvSearchResults','Select$2')">17-19-01-08-550</a></td><td>BEALE MOLLYE A</td><td>3731  LEASURE CT </td><td style="width:100px;">550</td><td style="wid

In [12]:
table_rows = adv_search_soup.find_all('tr', {'class': ['rowstyle', 'alternatingrowstyle']})
parcel_numbers = []
for columns in table_rows:
    parcel_number = list(columns.children)[1].string
    parcel_numbers.append(parcel_number)

In [13]:
parcel_numbers

['17-19-01-04-000',
 '17-19-01-04-003',
 '17-19-01-08-550',
 '17-19-01-11-550',
 '17-19-01-13-550',
 '17-19-01-14-551',
 '17-19-01-15-552',
 '17-19-01-16-553',
 '17-19-01-17-558',
 '17-19-01-18-559',
 '17-19-01-23-556',
 '17-19-01-27-564',
 '17-19-01-33-570',
 '17-19-01-35-572',
 '17-19-01-39-550',
 '17-19-01-43-550',
 '17-19-01-44-550',
 '17-19-01-48-550',
 '17-19-01-49-550',
 '17-19-01-56-550']

Using the small list of parcel numbers above, we will now show how to extract the details from the data pages:

In [14]:
parcel_data_driver = webdriver.PhantomJS(executable_path=os.path.join(PROJECT_PATH, 'resources', 'phantomjs'))

In [15]:
example_parcel = '17-19-01-17-558'
parcel_data_driver.get(urls['parcel-id-data-fmt'].format(parcel_id=example_parcel))

In [19]:
try:
    parcel_data_driver.execute_script("document.getElementById('ContentPlaceHolder1_btnDisclaimerAccept').click();")
except Exception as e:
    pass

parcel_data_driver.page_source

'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd"><html xmlns="http://www.w3.org/1999/xhtml"><head id="head"><meta http-equiv="X-UA-Compatible" content="IE=Edge"><title>\n\tMuskingum County, Ohio: Online Auditor - Property Data\n</title><meta http-equiv="content-type" content="text/html; charset=iso-8859-1"><meta http-equiv="content-language" content="en"><meta http-equiv="pragma" content="no-cache"><meta name="description" content="Online Real Estate and Property Search"><meta name="keywords"><meta name="robots" content="index,follow"><meta name="author" content="Kelly Menzel"><meta name="company" content="Digital Data Technologies, Inc."><meta name="copyright" content="Copyright ©2007"><link href="/favicon.ico" rel="Shortcut Icon"><link href="/stylesheets/print.css" rel="stylesheet" type="text/css" media="print"><link id="stylesheet" rel="stylesheet" type="text/css" href="/stylesheets/Canvas.css"><style type="tex

From the initial tab, called *Base*, we will need to fetch the address of the parcel. Once we have this, I will show how we will be finding the distance from the parcel address to the address of the dilapidated warehouse:

In [20]:
base_soup = BeautifulSoup(parcel_data_driver.page_source, 'html5lib')

In [21]:
street_address_id = 'ContentPlaceHolder1_Base_fvDataMailingAddress_MailingAddressLine2Label'
city_state_zip_id = 'ContentPlaceHolder1_Base_fvDataMailingAddress_MailingAddressLine3Label'

street_address_span = base_soup.find('span', {'id': [street_address_id]})
city_state_zip_span = base_soup.find('span', {'id': [city_state_zip_id]})

street = street_address_span.text
city, state, postalcode, *_ = city_state_zip_span.text.split(' ')

location_dict = {
    'street': street,
    'state': state,
    'postalcode': postalcode
}

In [120]:
location_geolocation = geolocator.geocode(location_dict)

address = location_geolocation.address
latitude = location_geolocation.latitude
longitude = location_geolocation.longitude

msg = '{}: {}'
print(msg.format('Address', address))
print(msg.format('Latitude', latitude))
print(msg.format('Longitude', longitude))

Address: Leasure Court South, Zanesville, Muskingum County, Ohio, 43701, United States of America
Latitude: 39.9954338
Longitude: -82.0333622


In [23]:
WAREHOUSE_LATLONG

(39.9252678, -82.0179831)

To measure the distance, we will use the Great Circle distance between two points on the globe:

In [121]:
latlong = (latitude, longitude)
distance_miles = great_circle(latlong, WAREHOUSE_LATLONG).miles

distance_msg = 'The distance from {} to our warehouse is {:.2f} miles.'
print(distance_msg.format(location_geolocation.address, distance_miles))

The distance from Leasure Court South, Zanesville, Muskingum County, Ohio, 43701, United States of America to our warehouse is 4.92 miles.


Now we need to navigate to the *Valuation* tab and scrape the necessary data for the most current valuation of the parcel:

In [25]:
parcel_data_driver.execute_script("__doPostBack('ctl00$ContentPlaceHolder1$mnuData','2')")

In [26]:
valuation_soup = BeautifulSoup(parcel_data_driver.page_source, 'html5lib')
valuation_soup.prettify()

'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml">\n <head id="head">\n  <meta content="IE=Edge" http-equiv="X-UA-Compatible"/>\n  <title>\n   Muskingum County, Ohio: Online Auditor - Property Data\n  </title>\n  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>\n  <meta content="en" http-equiv="content-language"/>\n  <meta content="no-cache" http-equiv="pragma"/>\n  <meta content="Online Real Estate and Property Search" name="description"/>\n  <meta name="keywords"/>\n  <meta content="index,follow" name="robots"/>\n  <meta content="Kelly Menzel" name="author"/>\n  <meta content="Digital Data Technologies, Inc." name="company"/>\n  <meta content="Copyright ©2007" name="copyright"/>\n  <link href="/favicon.ico" rel="Shortcut Icon"/>\n  <link href="/stylesheets/print.css" media="print" rel="stylesheet" type="text/css"/>\n  <link href="/stylesheets/Canvas

In [70]:
def get_numeric_appraised_value(span):
    try:
        appraised_value_str = span.text
    except AttributeError:
        print('[ERROR] span object is Nonetype')
    else:
        appraised_value = float(re.sub('[$,]', '', appraised_value_str))
        return appraised_value
    
appraised_total_value_id = 'ContentPlaceHolder1_Valuation_fvDataValuation_Label1'
appraised_total_value_span = valuation_soup.find('span', {'id': appraised_total_value_id})
appraised_total_value = get_numeric_appraised_value(appraised_total_value_span)

msg = 'The current appraised total value is : ${:,.0f}'
print(msg.format(appraised_total_value))

The current appraised total value is : $131,500


Finally, we will navigate to the *Residential* tab and scrape all of the relevant details about the residence, if one exists:

In [28]:
parcel_data_driver.execute_script("__doPostBack('ctl00$ContentPlaceHolder1$mnuData','8')")

In [30]:
residential_soup = BeautifulSoup(parcel_data_driver.page_source, 'html5lib')
residential_soup.prettify()

'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml">\n <head id="head">\n  <meta content="IE=Edge" http-equiv="X-UA-Compatible"/>\n  <title>\n   Muskingum County, Ohio: Online Auditor - Property Data\n  </title>\n  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>\n  <meta content="en" http-equiv="content-language"/>\n  <meta content="no-cache" http-equiv="pragma"/>\n  <meta content="Online Real Estate and Property Search" name="description"/>\n  <meta name="keywords"/>\n  <meta content="index,follow" name="robots"/>\n  <meta content="Kelly Menzel" name="author"/>\n  <meta content="Digital Data Technologies, Inc." name="company"/>\n  <meta content="Copyright ©2007" name="copyright"/>\n  <link href="/favicon.ico" rel="Shortcut Icon"/>\n  <link href="/stylesheets/print.css" media="print" rel="stylesheet" type="text/css"/>\n  <link href="/stylesheets/Canvas

In [31]:
def get_residential_data_by_id(css_id):
    """Returns the actual text from a span object in the 
    residential_soup object for the given CSS ID.
    
    """
    span = residential_soup.find('span', {'id': css_id})
    try:
        text = span.text
    except AttributeError:
        text = None
    return text
        
num_stories_id = 'ContentPlaceHolder1_Residential_fvDataResidential_Label2'
year_built_id = 'ContentPlaceHolder1_Residential_fvDataResidential_YearBuiltLabel'
num_bedrooms_id = 'ContentPlaceHolder1_Residential_fvDataResidential_NumberOfBedroomsLabel'
num_full_baths_id = 'ContentPlaceHolder1_Residential_fvDataResidential_NumberOfFullBathsLabel'
num_half_baths_id = 'ContentPlaceHolder1_Residential_fvDataResidential_NumberOfHalfBathsLabel'
finished_living_area_id = 'ContentPlaceHolder1_Residential_fvDataResidential_FinishedLivingAreaLabel'
basement_id = 'ContentPlaceHolder1_Residential_fvDataResidential_Label1'
basement_area_id = 'ContentPlaceHolder1_Residential_fvDataResidential_Label4'

num_stories = get_residential_data_by_id(num_stories_id)
year_built = get_residential_data_by_id(year_built_id)
num_bedrooms = get_residential_data_by_id(num_bedrooms_id)
num_full_baths = get_residential_data_by_id(num_full_baths_id)
num_half_baths = get_residential_data_by_id(num_half_baths_id)
finished_living_area = get_residential_data_by_id(finished_living_area_id)
basement = get_residential_data_by_id(basement_id)
basement_area = get_residential_data_by_id(basement_area_id)

residential_data_labels = [
    'NUM_STORIES', 'YEAR_BUILT', 'NUM_BEDROOMS', 'NUM_FULL_BATHS',
    'NUM_HALF_BATHS', 'FINISHED_LIVING_AREA_SQFT', 'BASEMENT',
    'BASEMENT_AREA_SQFT'
]

residential_data = [
    num_stories, year_built, num_bedrooms, num_full_baths,
    num_half_baths, finished_living_area, basement, basement_area
]

for label, data in zip(residential_data_labels, residential_data):
    print('{}: {}'.format(label, data))

NUM_STORIES: 1
YEAR_BUILT: 1999
NUM_BEDROOMS: 3
NUM_FULL_BATHS: 2
NUM_HALF_BATHS: 0
FINISHED_LIVING_AREA_SQFT: 1450
BASEMENT: NO
BASEMENT_AREA_SQFT: 0


## Data Aggregation

Our last step in this example is to aggregate the data and show it's final form:

In [124]:
data_rows = []
col_names = [
    'ADDRESS', 'LATITUDE', 'LONGITUDE', 'DIST_TO_WAREHOUSE',
    'CURRENT_APPRAISED_VALUE', 'NUM_STORIES', 'YEAR_BUILT', 
    'NUM_BEDROOMS', 'NUM_FULL_BATHS', 'NUM_HALF_BATHS', 
    'FINISHED_LIVING_AREA_SQFT', 'BASEMENT', 'BASEMENT_AREA_SQFT'
]

The following data_row would be created after scraping all of the necessary data for a parcel number. This data_row will be appended to a master list of rows, which will grow until the end of the scraping. Finally, this master list will be transformed into a pandas DataFrame so that it can easily be saved as a CSV file.

In [125]:
data_row = [
    address, latitude, longitude, distance_miles, appraised_total_value,
    num_stories, year_built, num_bedrooms, num_full_baths, num_half_baths, 
    finished_living_area, basement, basement_area
]

data_rows.append(data_row)

In [126]:
final_data_rows = np.array(data_rows)
final_df = pd.DataFrame(final_data_rows, columns=col_names)
final_df

,ADDRESS,LATITUDE,LONGITUDE,DIST_TO_WAREHOUSE,CURRENT_APPRAISED_VALUE,NUM_STORIES,YEAR_BUILT,NUM_BEDROOMS,NUM_FULL_BATHS,NUM_HALF_BATHS,FINISHED_LIVING_AREA_SQFT,BASEMENT,BASEMENT_AREA_SQFT
0,"Leasure Court South, Zanesville, Muskingum Cou...",39.9954338,-82.0333622,4.91732643848488,131500.0,1,1999,3,2,0,1450,NO,0
